In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier



In [2]:
#check datatypes
def check_dtypes2(dataframe):
    test =dataframe.dtypes
    x=0
    for i in range (len(dataframe.columns)):
        print("COL NAME: ",dataframe.columns[i])
        print("  TYPE: ",test[x])
        print("  NUMBER OF UNIQUE VALUES: ", dataframe[dataframe.columns[i]].nunique())
        print("  % OF UNIQUE VALUES: ", (len(pd.unique(dataframe[dataframe.columns[i]]))/len(dataframe[dataframe.columns[i]]))*100 )
        print("  Values: ",pd.unique(dataframe[dataframe.columns[i]]))
        print("  MODE: ",dataframe[dataframe.columns[i]].mode())
        print()
        x+=1
    return()
#dealing with null values

#null values info
def null_find(dataframe):
    null_df = pd.DataFrame(round(dataframe.isna().sum()/len(dataframe),4)*100)
    null_df = null_df.reset_index()
    null_df.columns = ['header','percentage_nulls']
    return(null_df)
           
#null values reduction
def null_reduction(dataframe,porcentage):
    null_df=null_find(dataframe)
    columns_drop = null_df[null_df['percentage_nulls']>porcentage]['header']
    return dataframe.drop(columns_drop,axis=1)

# print non zero null columns

def print_nan(dataframe):
    null_df = null_find(dataframe)
    null_df = null_df[null_df['percentage_nulls']>0]
    print(null_df)
    return()


#scaler
def scaleColumns(df, cols_to_scale):
    scaler = preprocessing.StandardScaler()

    for col in cols_to_scale:
        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(df[col])),columns=[col])
    return df


Read that data into Python and call the dataframe churnData.

In [3]:
categorical = pd.read_csv('./files_for_lab/categorical.csv')
numerical = pd.read_csv('./files_for_lab/numerical.csv')
target = pd.read_csv('./files_for_lab/target.csv')


In [4]:
check_dtypes2(categorical)

COL NAME:  STATE
  TYPE:  object
  NUMBER OF UNIQUE VALUES:  12
  % OF UNIQUE VALUES:  0.012577034335303737
  Values:  ['IL' 'CA' 'NC' 'FL' 'other' 'IN' 'MI' 'MO' 'TX' 'WA' 'WI' 'GA']
  MODE:  0    other
dtype: object

COL NAME:  CLUSTER
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  53
  % OF UNIQUE VALUES:  0.05554856831425817
  Values:  [36 14 43 44 16 40 39 45 35 53 17 51  2 20 27 12 22 13  8 25 15 42 11 28
 18 24 34  5 31 32 46  3 50  7 37 10 38 21  9 29 30  4 41 49 23 33  1 47
 26 48 19  6 52]
  MODE:  0    40
dtype: int64

COL NAME:  HOMEOWNR
  TYPE:  object
  NUMBER OF UNIQUE VALUES:  2
  % OF UNIQUE VALUES:  0.002096172389217289
  Values:  ['H' 'U']
  MODE:  0    H
dtype: object

COL NAME:  GENDER
  TYPE:  object
  NUMBER OF UNIQUE VALUES:  3
  % OF UNIQUE VALUES:  0.0031442585838259342
  Values:  ['F' 'M' 'other']
  MODE:  0    F
dtype: object

COL NAME:  DATASRCE
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  3
  % OF UNIQUE VALUES:  0.0031442585838259342
  Values:  [3 1 2]
  MODE: 

()

In [5]:
check_dtypes2(numerical)

COL NAME:  TCODE
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  55
  % OF UNIQUE VALUES:  0.05764474070347545
  Values:  [    0     1     2    28     3  1002    42     4    18   980    14 28028
    72    22 13002    23    45    24  4002    30    13   202   136 72002
    96   116   100     6  4004 39002    61    47    36   228 14002  6400
    40    25    21    94    12 58002   134 18002    38     9    76    50
    27    93    17     7    44 24002 22002]
  MODE:  0    0
dtype: int64

COL NAME:  AGE
  TYPE:  float64
  NUMBER OF UNIQUE VALUES:  97
  % OF UNIQUE VALUES:  0.10166436087703852
  Values:  [60.         46.         61.61164927 70.         78.         38.
 65.         75.         72.         44.         62.         82.
 74.         54.         42.         84.         49.         69.
 88.         30.         51.         66.         61.         68.
 83.         45.         40.         48.         34.         80.
 36.         50.         41.         71.         47.         33.
 63.      

  % OF UNIQUE VALUES:  0.08908732654173479
  Values:  [ 0  1  3  2  8  6 14  9  4  7 11  5 31 99 18 10 95 96 13 24 28 22 17 15
 21 48 12 51 16 25 32 30 92 71 68 19 34 29 98 20 27 89 90 80 67 38 23 45
 75 33 62 55 26 44 47 35 36 41 43 91 73 37 59 53 54 77 50 85 97 46 78 94
 52 93 74 39 65 42 49 70 72 88 66 40 63]
  MODE:  0    0
dtype: int64

COL NAME:  ETH4
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  96
  % OF UNIQUE VALUES:  0.1006162746824299
  Values:  [ 0 31 11  1 16  5  2 10  3 27  4 15 39 19  7 20 12  9 17 14  8  6 35 55
 36 13 50 22 25 23 51 83 80 61 32 34 46 29 18 37 58 54 43 28 33 91 85 24
 41 89 40 21 30 71 48 49 77 90 26 67 42 68 74 47 65 45 75 87 53 44 84 38
 60 66 56 64 63 92 69 52 70 88 59 95 57 78 76 99 73 86 72 62 93 79 81 82]
  MODE:  0    0
dtype: int64

COL NAME:  ETH5
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  100
  % OF UNIQUE VALUES:  0.10480861946086446
  Values:  [11  6  2 32  1  0  5 12  4 37 16  3 13 27 23 30  8 14 25 18 60 28  9 10
  7 17 15 19 45 98 50 20 34 

  MODE:  0    20
dtype: int64

COL NAME:  AGEC3
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  57
  % OF UNIQUE VALUES:  0.05974091309269274
  Values:  [18 33 19 17 13 24 26 21 16 20 23 27 29 22 39 34 15 25  0  6 31 11 32  5
  4 12  9 10  8 44 28 14 30 43  7 35 40 36  3 42  1 41 37 38 46  2 48 45
 63 50 47 58 51 49 99 53 60]
  MODE:  0    19
dtype: int64

COL NAME:  AGEC4
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  43
  % OF UNIQUE VALUES:  0.04506770636817172
  Values:  [17 23 16 15 10 11 14  8 13 20 21 18 12  9 22  0  3 19 28  5 24  4  6 25
  2  7  1 27 31 26 33 32 30 29 37 34 42 50 41 99 35 40 36]
  MODE:  0    14
dtype: int64

COL NAME:  AGEC5
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  43
  % OF UNIQUE VALUES:  0.04506770636817172
  Values:  [13 10 12 20  8 14 11  9 17 15 21  5  6  0 16 19  7  2 22 24 18  4  3 23
 27  1 25 26 32 30 28 29 36 39 33 34 31 40 38 37 99 35 50]
  MODE:  0    12
dtype: int64

COL NAME:  AGEC6
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  63
  % OF UNIQUE VALUES:  

  Values:  [95 99 69 83 98 66 73 62 88 72 22 80 71 81 79 85 70 49 67 94 96 34 92 78
 52  8 63  0  5 33  6 84 93 58 50 89 86 97 91 75 30 74 51 23 77 39  1  9
 10 53 46 68 21  3 42 57 54 87 13 64  4 61 48 56 41 90 60 47 59 25 40 44
 45 76 15 65 43 32 55 20 26 11  7 16 82 37 17 38 28 31 36 27 18 19  2 12
 14 24 29 35]
  MODE:  0    99
dtype: int64

COL NAME:  DW3
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  70
  % OF UNIQUE VALUES:  0.07336603362260513
  Values:  [ 2  0  1  3 14  4  7  6 18  5  9  8 12 22 25 15 11 19 17 10 29 13 43 33
 24 20 26 16 52 28 48 21 23 27 37 34 38 56 54 40 46 66 32 39 30 41 45 36
 65 31 53 51 61 55 50 68 70 42 49 47 35 44 60 58 59 57 67 63 99 62]
  MODE:  0    0
dtype: int64

COL NAME:  DW4
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  100
  % OF UNIQUE VALUES:  0.10480861946086446
  Values:  [ 2  0  6  4 23 16  7 22 74  3 32 18 21 10 28  5  1 26 44 91 12 45 57 14
  8  9 31 11 30 34 66 47 20 59 95 27 49 46 54 19 56 42 29 99 48 35 39 24
 77 51 13 73 50 68 52 17 76 38 

  NUMBER OF UNIQUE VALUES:  64
  % OF UNIQUE VALUES:  0.06707751645495325
  Values:  [ 5  3 17  6  7 10  1  9  4 13  8  2  0 11 18 15 27 16 12 19 24 30 14 21
 37 48 23 29 22 20 26 38 62 25 34 28 36 31 35 99 43 39 55 44 32 33 45 41
 40 46 54 53 47 50 49 51 42 59 56 58 60 52 64 83]
  MODE:  0    4
dtype: int64

COL NAME:  HHD10
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  83
  % OF UNIQUE VALUES:  0.08699115415251751
  Values:  [ 9  5 11 21 12 13 15 30 10  6 23  8 18  4 17 28  0 16  7 14 19 22 36 20
 25 60 27  3 50  2 29 24 55 46 33 26 39 32 34 41 45 48 31 37 38 43 40 47
 35 42 44 62 67 52 64 56 49  1 54 58 53 99 93 69 51 57 59 74 82 61 77 81
 70 72 65 88 66 86 80 63 83 71 68]
  MODE:  0    11
dtype: int64

COL NAME:  HHD11
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  82
  % OF UNIQUE VALUES:  0.08594306795790885
  Values:  [15  4 17 12 23 14 20  7 19 18 24 53 13 22 25 16 21 10 26 39  0 71 30  8
  6 11 29 34 28 40 27 42 36  5 38  1 31 35  9 37  2 33 43 32 46 41  3 44
 45 47 49 55 54 52 51 58

...

  NUMBER OF UNIQUE VALUES:  63
  % OF UNIQUE VALUES:  0.06602943026034461
  Values:  [10  5 20 12 14 17  3 11 18  4 16  6 13 23 24 19  2  1  9  0  7 22 28  8
 15 25 21 36 34 26 27 31 29 30 33 32 39 38 82 37 55 41 35 40 44 43 99 52
 47 45 51 58 62 48 53 49 46 42 50 56 57 61 54]
  MODE:  0    10
dtype: int64

COL NAME:  EC4
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  80
  % OF UNIQUE VALUES:  0.08384689556869157
  Values:  [39 12 28 14 30 27 44 49 47 38 17 40 36 16 42 37 24 23 22 32 21  0 34 31
 29 43 35 25 51 41 20  9 15 33 26 48  6 11 19 52 54 53 18 13 10  8  4  5
 46 56 50 45  7 58  1 71 55  2  3 63 57 60 59 99 61 73 66 72 62 69 67 75
 65 70 64 90 88 78 74 68]
  MODE:  0    31
dtype: int64

COL NAME:  EC5
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  69
  % OF UNIQUE VALUES:  0.07231794742799648
  Values:  [21 16 27 13 19 17 20 15 24 12 26 31 18 14 23 25 36  0 29 10  9 22 28 30
  5 11 35 32 33 41 34  6  7  8 38 37 40 47  3 43  4 44 49  2 60 39 50 48
 42 51 52 61 46  1 99 45 64 74 54 53 75

  MODE:  0    4
dtype: int64

COL NAME:  ANC8
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  42
  % OF UNIQUE VALUES:  0.044019620173563076
  Values:  [ 1  3  0  6  2  4  8  7  5  9 12 13 10 18 19 16 14 11 17 34 15 23 31 25
 40 21 45 20 26 22 27 29 28 30 33 32 55 37 43 24 39 35]
  MODE:  0    0
dtype: int64

COL NAME:  ANC9
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  52
  % OF UNIQUE VALUES:  0.05450048211964952
  Values:  [ 0  1  5  2  4  3  8  7 22  6  9 16 11 24 10 54 29 28 12 18 49 13 14 15
 17 23 20 32 27 30 31 47 44 21 35 25 48 36 26 19 38 37 42 39 57 52 41 43
 55 51 68 50]
  MODE:  0    0
dtype: int64

COL NAME:  ANC10
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  60
  % OF UNIQUE VALUES:  0.06288517167651868
  Values:  [ 0  1 11  3  5  9  2  8  4  6 15 13 10 18 17 12 39  7 21 16 32 22 20 14
 27 33 31 50 19 26 29 30 65 23 24 44 37 25 36 28 46 40 71 48 38 41 63 47
 58 45 42 53 57 35 55 99 74 34 52 43]
  MODE:  0    0
dtype: int64

COL NAME:  ANC11
  TYPE:  int64
  NUMBER OF UNIQUE VALUES

  MODE:  0    99
dtype: int64

COL NAME:  HC8
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  100
  % OF UNIQUE VALUES:  0.10480861946086446
  Values:  [46  0 31 55 67 12  8 78 37 69 99 53 23 26 13 98 54 29  1 50 38 21 70 17
 62  5 91 28  3 90 11 51  7 42 47 39 36 80 15 44 86 45 30  6 87 18 32 64
 58 10 81 14 84 33 34 61 92 25 63 35 19  2 85 94 20 95 16 93 75 22 59 56
 66  4 96 71 89 57 40 73 49  9 52 24 60 41 43 82 74 88 83 79 48 77 97 72
 65 27 68 76]
  MODE:  0    0
dtype: int64

COL NAME:  HC9
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  87
  % OF UNIQUE VALUES:  0.09118349893095208
  Values:  [ 0  1  3 16 12 11 22 41  9 36  5  7  8  2 59 21 39 35  4 14 25  6 10 20
 66 27 19 13 15 24 63 44 18 49 64 67 42 23 31 33 53 26 43 32 28 34 45 30
 54 37 71 29 17 40 73 38 52 60 58 47 51 50 48 65 89 74 79 75 57 55 81 46
 77 69 72 62 56 70 82 68 61 76 84 88 78 80 90]
  MODE:  0    0
dtype: int64

COL NAME:  HC10
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  53
  % OF UNIQUE VALUES:  0.05554856831425817
 

  NUMBER OF UNIQUE VALUES:  191
  % OF UNIQUE VALUES:  0.2001844631702511
  Values:  [5.0000e+00 1.0000e+01 2.0000e+00 3.0000e+00 2.0000e+01 4.0000e+00
 1.5000e+01 2.5000e+01 5.0000e+01 1.0000e+00 2.3000e+01 3.5000e+00
 3.0000e+01 9.0000e+00 6.0000e+00 0.0000e+00 7.0000e+00 1.2500e+00
 5.0000e-01 2.4000e+01 7.0000e-02 6.0000e-02 1.3000e+01 1.2000e+01
 4.5000e+01 1.0000e+02 8.0000e+00 7.6500e+00 1.8000e+01 2.8000e+01
 1.6000e+01 8.0000e-02 2.5000e+00 1.5000e+00 1.5300e+00 3.0200e+00
 1.2500e+01 4.0000e+01 1.0500e+00 5.3000e+01 3.6000e+01 3.5000e+01
 4.2730e+01 8.8140e+01 5.0700e+00 3.4000e+01 2.0200e+00 8.5000e+00
 5.9000e+01 1.7000e+01 1.0300e+02 1.6080e+01 4.9400e+00 1.1000e+01
 2.0500e+00 2.0000e-02 1.4000e+01 6.0000e+01 1.9000e+01 1.5000e+02
 9.0000e-02 1.0000e-02 5.0000e-02 1.3400e+01 6.5000e+00 1.3500e+01
 1.6090e+01 1.0100e+00 1.1700e+00 2.2000e+01 2.2500e+00 2.0000e+02
 3.5200e+00 5.5340e+01 4.5000e+00 3.0500e+00 3.0000e-02 1.0200e+00
 6.8600e+00 5.5000e+00 2.1000e+00 5.5000e+01

()

In [6]:
check_dtypes2(target)

COL NAME:  TARGET_B
  TYPE:  int64
  NUMBER OF UNIQUE VALUES:  2
  % OF UNIQUE VALUES:  0.002096172389217289
  Values:  [0 1]
  MODE:  0    0
dtype: int64

COL NAME:  TARGET_D
  TYPE:  float64
  NUMBER OF UNIQUE VALUES:  71
  % OF UNIQUE VALUES:  0.07441411981721377
  Values:  [  0.     4.     7.     5.    13.    10.    25.     8.    20.    16.
  26.    15.     3.    60.    23.     6.    11.    18.    16.87   2.5
  50.    10.7   35.    14.    21.    17.   100.     2.    12.    19.
  40.    38.    45.    30.    12.5    7.5    1.     9.    36.    22.
  24.    41.    51.    28.    32.    43.    31.    33.    75.    47.
  27.    37.    42.    18.25  34.    13.92  29.   200.    46.    44.
  53.     5.25  95.    17.5   48.   101.   150.     4.5   55.   102.
  44.21]
  MODE:  0    0.0
dtype: float64



()

In [7]:
data = pd.concat([numerical,categorical,target['TARGET_B']], axis=1)

In [8]:
null_find(data)

,header,percentage_nulls
0,TCODE,0.0
1,AGE,0.0
2,INCOME,0.0
3,WEALTH1,0.0
4,HIT,0.0
...,...,...
333,LASTDATE_YR,0.0
334,LASTDATE_MM,0.0
335,FIRSTDATE_YR,0.0
336,FIRSTDATE_MM,0.0


# Lab | Random Forests

For this lab, you will be using the CSV files provided in the files_for_lab folder.

In [9]:
Xf=data.drop(['TARGET_B'], axis=1)
yf=data['TARGET_B']
Xf_num=Xf.select_dtypes(include = np.number)
Xf_cat = pd.get_dummies(Xf.select_dtypes(include = 'object'), drop_first=True)
Xf=pd.concat([Xf_num, Xf_cat], axis=1)

# Apply the Random Forests algorithm but this time only by upscaling the data using SMOTE.

Apply SMOTE for upsampling the data

Note that since SMOTE works on numerical data only, we will first encode the categorical variables in this case.

In [10]:
from imblearn.over_sampling import SMOTE



data = pd.concat([Xf,yf],axis=1)
smote = SMOTE()
y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis=1)
X_sm, y_sm = smote.fit_resample(X, y)

unique, counts = np.unique(y_sm, return_counts=True)
print(np.asarray((unique, counts)).T)

[[    0 90569]
 [    1 90569]]


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.3, random_state=42)

Testing with a tree depth of 2

In [12]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)


RandomForestClassifier(max_depth=2, random_state=0)

In [13]:
predicted = clf.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.85      0.76      0.80     27089
           1       0.79      0.87      0.83     27253

    accuracy                           0.82     54342
   macro avg       0.82      0.82      0.81     54342
weighted avg       0.82      0.82      0.81     54342



X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.3, random_state=42)

Testing with a tree depth of 4

In [14]:
clf = RandomForestClassifier(max_depth=4, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, random_state=0)

In [15]:
predicted = clf.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.88      0.83      0.86     27089
           1       0.84      0.89      0.87     27253

    accuracy                           0.86     54342
   macro avg       0.86      0.86      0.86     54342
weighted avg       0.86      0.86      0.86     54342

